# Statistical Downscaling and Bias-Adjustment

`xclim` provides tools and utilities to ease the bias-adjustement process through its `xclim.sdba` module. Adjustment algorithms all conform to the `train` - `adjust` scheme, formalized within `Adjustment` classes. Given a reference time series (ref), historical simulations (hist) and simulations to be adjusted (sim), any bias-adjustment method would be applied by first estimating the adjustment factors between the historical simulation and the observations series, and then applying these factors to `sim`, which could be a future simulation.

A very simple "Quantile Mapping" approach is available through the "Empirical Quantile Mapping" object.

In [ ]:
import netCDF4  # Needed for scipy.io.netcdf
import numpy as np
import xarray as xr
import cftime
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (11, 5)

# Create toy data to explore bias adjustment, here fake temperature timeseries
t = xr.cftime_range('2000-01-01', '2030-12-31', freq='D', calendar='noleap')
ref = xr.DataArray((-20 * np.cos(2 * np.pi * t.dayofyear / 365) + 2 * np.random.random_sample((t.size,)) + 273.15
                    + 0.1 * (t - t[0]).days / 365),  # "warming" of 1K per decade,
                   dims=('time',), coords={'time': t}, attrs={'units': 'K'})
sim = xr.DataArray((-18 * np.cos(2 * np.pi * t.dayofyear / 365) + 2 * np.random.random_sample((t.size,)) + 273.15
                    + 0.11 * (t - t[0]).days / 365),  # "warming" of 1.1K per decade
                   dims=('time',), coords={'time': t}, attrs={'units': 'K'})

ref = ref.sel(time=slice(None, '2015-01-01'))
hist = sim.sel(time=slice(None, '2015-01-01'))

ref.plot(label='Reference')
sim.plot(label='Model')
plt.legend()

In [ ]:
from xclim import sdba

QM = sdba.EmpiricalQuantileMapping(nquantiles=15, group='time', kind='+')
QM.train(ref, hist)
scen = QM.adjust(sim, extrapolation='constant', interp='nearest')

ref.groupby('time.dayofyear').mean().plot(label='Reference')
hist.groupby('time.dayofyear').mean().plot(label='Model - biased')
scen.sel(time=slice('2000', '2015')).groupby('time.dayofyear').mean().plot(label='Model - adjusted - 2000-15', linestyle='--')
scen.sel(time=slice('2015', '2030')).groupby('time.dayofyear').mean().plot(label='Model - adjusted - 2015-30', linestyle='--')
plt.legend()

In the previous example, a simple Quantile Mapping algorithm was used with 15 quantiles and one group of values. The model performs well, but our toy data is also quite smooth and well-behaved so this is not surprising. A more complex example could have biais distribution varying strongly across months. To perform the adjustment with different factors for each months, one can pass `group='time.month'`. Moreover, to reduce the risk of sharp change in the adjustment at the interface of the months, `interp='linear'` can be passed to `adjust` and the adjustment factors will be interpolated linearly. Ex: the factors for the 1st of May will be the average of those for april and those for may. This option is currently only implemented for monthly grouping.

In [ ]:
QM_mo = sdba.EmpiricalQuantileMapping(nquantiles=15, group='time.month', kind='+')
QM_mo.train(ref, hist)
scen = QM_mo.adjust(sim, extrapolation='constant', interp='linear')

ref.groupby('time.dayofyear').mean().plot(label='Reference')
hist.groupby('time.dayofyear').mean().plot(label='Model - biased')
scen.sel(time=slice('2000', '2015')).groupby('time.dayofyear').mean().plot(label='Model - adjusted - 2000-15', linestyle='--')
scen.sel(time=slice('2015', '2030')).groupby('time.dayofyear').mean().plot(label='Model - adjusted - 2015-30', linestyle='--')
plt.legend()

The training data (here the adjustment factors) is available for inspection in the `ds` attribute of the adjustment object.

In [ ]:
QM_mo.ds

In [ ]:
QM_mo.ds.af.plot()

## Grouping

For basic time period grouping (months, day of year, season), passing a string to the methods needing it is sufficient. Most methods acting on grouped data also accept a `window` int argument to pad the groups with data from adjacent ones. Units of `window` are the sampling frequency of the main grouping dimension (usually `time`). For more complex grouping, one can pass a :py:class:`xclim.sdba.base.Grouper` directly.

Example here with another, simpler, adjustment method. Here we want `sim` to be scaled so that its mean fits the one of `ref`. Scaling factors are to be computed separately for each day of the year, but including 15 days on either side of the day. This means that the factor for the 1st of May is computed including all values from the 16th of April to the 15th of May (of all years).

In [ ]:
group = sdba.Grouper('time.dayofyear', window=31)
QM_doy = sdba.Scaling(group=group, kind='+')
QM_doy.train(ref, hist)
scen = QM_doy.adjust(sim)

ref.groupby('time.dayofyear').mean().plot(label='Reference')
hist.groupby('time.dayofyear').mean().plot(label='Model - biased')
scen.sel(time=slice('2000', '2015')).groupby('time.dayofyear').mean().plot(label='Model - adjusted - 2000-15', linestyle='--')
scen.sel(time=slice('2015', '2030')).groupby('time.dayofyear').mean().plot(label='Model - adjusted - 2015-30', linestyle='--')
plt.legend()

In [ ]:
QM_doy.ds.af.plot()

## Modular approach

The `sdba` module adopts a modular approach instead of implementing published and named methods directly.
A generic bias adjustment process is laid out as follows:

- preprocessing on `ref`, `hist` and `sim` (using methods in `xclim.sdba.processing` or `xclim.sdba.detrending`)
- creating the adjustment object `Adj = Adjustment(**kwargs)` (from `xclim.sdba.adjustment`)
- training `Adj.train(obs, sim)`
- adjustment `scen = Adj.adjust(sim, **kwargs)`
- post-processing on `scen` (for example: re-trending)

The train-adjust approach allows to inspect the trained adjustment object. The training information is stored in the underlying `Adj.ds` dataset and always has a `af` variable with the adjustment factors. Its layout and the other available variables vary between the different algorithm, refer to their part of the API docs.

Parameters needed by the training and the adjustment are saved to the `Adj.ds` dataset as a  `adj_params` attribute. Other parameters, those only needed by the adjustment are passed in the `adjust` call and written to the history attribute in the output scenario dataarray.

### First example : pr and frequency adaptation

The next example generates fake precipitation data and adjusts the `sim` timeseries but also adds a step where the dry-day frequency of `hist` is adapted so that is fits the one of `ref`. This ensures well-behaved adjustment factors for the smaller quantiles. Note also that we are passing `kind='*'` to use the multiplicative mode. Adjustment factors will be multiplied/divided instead of being added/substracted.

In [ ]:
vals = np.random.randint(0, 1000, size=(t.size,)) / 100
vals_ref = (4 ** np.where(vals < 9, vals/ 100, vals)) / 3e6
vals_sim = (1 + 0.1 * np.random.random_sample((t.size,))) * (4 ** np.where(vals < 9.5, vals/ 100, vals)) / 3e6

pr_ref = xr.DataArray(vals_ref, coords={"time": t}, dims=("time",), attrs={'units': 'mm/day'})
pr_ref = pr_ref.sel(time=slice('2000', '2015'))
pr_sim = xr.DataArray(vals_sim, coords={"time": t}, dims=("time",), attrs={'units': 'mm/day'})
pr_hist = pr_sim.sel(time=slice('2000', '2015'))

pr_ref.plot(alpha=0.9, label='Reference')
pr_sim.plot(alpha=0.7, label='Model')
plt.legend()

In [ ]:
# 1st try without adapt_freq
QM = sdba.EmpiricalQuantileMapping(nquantiles=15, kind='*', group='time')
QM.train(pr_ref, pr_hist)
scen = QM.adjust(pr_sim)

pr_ref.sel(time='2010').plot(alpha=0.9, label='Reference')
pr_hist.sel(time='2010').plot(alpha=0.7, label='Model - biased')
scen.sel(time='2010').plot(alpha=0.6, label='Model - adjusted')

In the figure above, `scen` has small peaks where `sim` is 0. This problem originates from the fact that there are more "dry days" (days with almost no precipitation) in `hist` than in `ref`. The next example works around the problem using frequency-adaptation, as described in [Themeßl et al. (2010)](https://doi.org/10.1007/s10584-011-0224-4).

In [ ]:
# 2nd try with adapt_freq
ds_ad = sdba.processing.adapt_freq(sim=pr_hist, ref=pr_ref, thresh=0.05)
QM_ad = sdba.EmpiricalQuantileMapping(nquantiles=15, kind='*', group='time')
QM_ad.train(pr_ref, ds_ad.sim_ad)
scen_ad = QM_ad.adjust(pr_sim)

pr_ref.sel(time='2010').plot(alpha=0.9, label='Reference')
pr_sim.sel(time='2010').plot(alpha=0.7, label='Model - biased')
scen_ad.sel(time='2010').plot(alpha=0.6, label='Model - adjusted')

### Second example: tas and detrending

The next example reuses the fake temperature timeseries generated at the beginning and applies the same QM adjustment method. However, for a better adjustment, we will scale sim to ref and then detrend the series, assuming the trend is linear. When `sim` (or `sim_scl`) is detrended, its values are now anomalies, so we need to normalize `ref` and `hist` so we can compare similar values.

This process is detailed here to show how the sdba module should be used in custom adjustment processes, but this specific method also exists as `sdba.DetrendedQuantileMapping` and is based on [Cannon et al. 2015](https://doi.org/10.1175/JCLI-D-14-00754.1). However, `DetrendedQuantileMapping` normalizes over a `time.dayofyear` group, regardless of what is passed in the  `group` argument. As done here, it is anyway recommended to use `dayofyear` groups when normalizing, especially for variables with strong seasonal variations.

In [ ]:
doy_win31 = sdba.Grouper('time.dayofyear', window=15)
Sca = sdba.Scaling(group=doy_win31, kind='+')
Sca.train(ref, hist)
sim_scl = Sca.adjust(sim)

detrender = sdba.detrending.PolyDetrend(degree=1, group='time.dayofyear', kind='+')
sim_fit = detrender.fit(sim_scl)
sim_detrended = sim_fit.detrend(sim_scl)

ref_n = sdba.processing.normalize(ref, group=doy_win31, kind='+')
hist_n = sdba.processing.normalize(hist, group=doy_win31, kind='+')

QM = sdba.EmpiricalQuantileMapping(nquantiles=15, group='time.month', kind='+')
QM.train(ref_n, hist_n)
scen_detrended = QM.adjust(sim_detrended, extrapolation='constant', interp='nearest')
scen = sim_fit.retrend(scen_detrended)


ref.groupby('time.dayofyear').mean().plot(label='Reference')
sim.groupby('time.dayofyear').mean().plot(label='Model - biased')
scen.sel(time=slice('2000', '2015')).groupby('time.dayofyear').mean().plot(label='Model - adjusted - 2000-15', linestyle='--')
scen.sel(time=slice('2015', '2030')).groupby('time.dayofyear').mean().plot(label='Model - adjusted - 2015-30', linestyle='--')
plt.legend()

## Optimization with dask

Adjustment process can be very heavy as they are made of large amounts of small operation and often needs to be computed over large regions. Using small groupings (like `time.dayofyear`) adds precision and robustness, but also decuplates the load and computing complexity. A good first read on this are xarray's [Optimization tips](http://xarray.pydata.org/en/stable/dask.html#optimization-tips).

Some additionnal tips:

* When saving a file with `to_netcdf`, setting option `unlimited_dims` with a list of dimension names can force xarray to write the data using chunks on disk, instead of one contiguous array. Command-line tool `ncdump -sh` can give information on how those chunks are oriented on disk and the optimal `chunks={...}` choice can be inferred.
* Most adjustment method will need to perform operation on the whole `time` coordinate, so it is best to optimize chunking along the other dimensions.
* One of the main bottleneck for adjustments with small groups is that dask needs to build and optimize an enormous task graph. In order to ease that process and reduce the number of recalculations, given that the training dataset fits in memory, one could call `Adjustment.ds.load()` to trigger the computation and store the result as a `np.array`. For very large tasks, one could write the training dataset to disk and then reload them into the `Adjustment` object.
* Consider using `engine="h5netcdf"` in `open_[mf]dataset` when possible. Compatibility of a file can be found using the `is_hdf5()` method of the `h5py` module.

## AN EXAMPLE IS MISSING